In [38]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

sessions_cleaned <- sessions |> mutate(email = as_factor(hashedEmail)) |> select(-hashedEmail)

players_sorted <- players |> mutate(email = as_factor(hashedEmail))
sessions_named <- sessions_cleaned |> left_join(players_sorted, by = "email") |> select(-hashedEmail)

sessions_final <- sessions_named |> filter(gender == "Female") |> mutate(start = dmy_hm(start_time), 
end = dmy_hm(end_time), start_date = as_date(start), start_time = format(start, "%H:%M"), 
end_date = as_date(end_time), end_time = format(end, "%H:%M")) |>
select(-start, -end, -email, -start_date, -end_date)
sessions <- sessions_final |> filter(gender == "Female") |>  mutate(start = as.POSIXct(paste("2025-01-01", start_time), format = "%Y-%m-%d %H:%M"),
end = as.POSIXct(paste("2025-01-01", end_time), format = "%Y-%m-%d %H:%M")) |> mutate(end = if_else(end < start, end + lubridate::days(1), end)) |> 
mutate(session_minutes = as.numeric(difftime(end, start, units = "mins"))) |> select(-start, -end, -original_start_time, -original_end_time) |>
group_by(name) |> mutate(total_minutes = sum(session_minutes))
sessions

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


start_time,end_time,experience,subscribe,played_hours,name,gender,Age,session_minutes,total_minutes
<chr>,<chr>,<chr>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
17:34,17:57,Amateur,TRUE,150.0,Delara,Female,16,23,9947
02:13,02:30,Regular,TRUE,178.2,Piper,Female,19,17,11827
15:15,15:43,Amateur,TRUE,150.0,Delara,Female,16,28,9947
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
15:05,17:02,Amateur,TRUE,150.0,Delara,Female,16,117,9947
02:30,04:22,Amateur,TRUE,150.0,Delara,Female,16,112,9947
04:08,04:19,Veteran,TRUE,1.6,Lane,Female,23,11,115
